# Antialising contrast-corrected Gabor noise

In [ ]:
using MVW, MsgPack, Images, Plots

In [ ]:
# Load the parameters from a file
function loadparams(c, file)
    for (k, v) in MsgPack.unpack(read(file))
        try
            setparam(c, k, v)
        catch err
            println(err)
        end
    end
end

# Save the current parameters of the GN shader to a file
function saveparams(c, file)
    params = map(getparams(c)) do p
        (p["s_name"], p["value"])
    end
    write(file, MsgPack.pack(params))
end

In [ ]:
c = spawn("../../build", ["-g", "../models/mcguire/bunny/bunny.obj", "-s", "glsl/gabor-noise-solid.glsl", "-p", "glsl/pp-identity.glsl", "-m"])

In [ ]:
loadparams(c, "sphere.bin")

In [ ]:
noise(c) = Gray.(channelview(getframe(c; size=(512, 512)))[1, :, :])
function alln(c; target="")
    channels = channelview(getframe(c; size=(1024, 1024), target=target))
    map(1:4) do i
        Gray.(channels[i, :, :])
    end
end

In [ ]:
using FFTW, LinearAlgebra, Interact, Statistics

In [ ]:
@manipulate for f0 = 1:1:200, w0x = 0:0.1:(2*pi), w0y = 0:0.1:(2*pi)
    setparam(c, "gF0", f0)
    setparam(c, "gTilesize", 0.1)
    setparam(c, "gW0", [w0x, w0y])
    img = channelview(getframe(c))[3, :, :]
    img_spectrum = log.(norm.(fftshift(fft(img)))^2)
    Gray.(img_spectrum ./ maximum(img_spectrum))
end

In [ ]:
imgs=alln(c; target="geometry")

In [ ]:
heatmap(channelview(imgs[2]), aspectratio=1, yflip=true)

In [ ]:
heatmap(channelview(imgs[3]), aspectratio=1, yflip=true)

In [ ]:
finalize(c)

In [ ]:
c = connect()